In [1]:
!pip install -U aifactory
!pip install -U gdown

  Obtaining dependency information for aifactory from https://files.pythonhosted.org/packages/c7/9d/776e3eca285dbc0e55695a8199f37325223979777a1dcd9d86a42b043743/aifactory-1.7.3-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=404808506ec81a488b36bfea864ae00b5a04d4aaebbf12e61bb515d59e91e6a7
  Stored in directory: c:\users\dustn\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063



  Attempting uninstall: gdown
    Found existing installation: gdown 4.5.1
    Uninstalling gdown-4.5.1:
      Successfully uninstalled gdown-4.5.1


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from PIL import Image
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [4]:
train_df = pd.read_csv("train.csv")
train_df

,ImagePath,Label
0,dataset/train/daisy/16020253176_60f2a6a5ca_n.jpg,daisy
1,dataset/train/daisy/267148092_4bb874af58.jpg,daisy
2,dataset/train/daisy/4318007511_e9f4311936_n.jpg,daisy
3,dataset/train/daisy/9595857626_979c45e5bf_n.jpg,daisy
4,dataset/train/daisy/3699235066_fc09a02dfe_m.jpg,daisy
...,...,...
2564,dataset/train/roses/14107161906_5737e0e4ec.jpg,roses
2565,dataset/train/roses/12243069253_e512464095_n.jpg,roses
2566,dataset/train/roses/2053476785_c162a3e358.jpg,roses
2567,dataset/train/roses/8747396730_966149e6fe_n.jpg,roses


In [5]:
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)
print(train_df.shape)
print(valid_df.shape)

(2055, 2)
(514, 2)


In [6]:
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [8]:
def image_file_to_array_fn_for_train_df(df):
    images = []
    baseURL = './train'
    for path in tqdm(df['ImagePath']):
        path = baseURL + path[13:]
        image = Image.open(path).resize((180, 180))
        image = np.array(image, np.uint8)
        images.append(image)
    return images

In [9]:
train_images = image_file_to_array_fn_for_train_df(train_df)
valid_images = image_file_to_array_fn_for_train_df(valid_df)
print(train_images[0].shape)

  0%|          | 0/2055 [00:00<?, ?it/s]

  0%|          | 0/514 [00:00<?, ?it/s]

(180, 180, 3)


In [10]:
label_dict = {
    # ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
    'daisy': 0,
    'dandelion': 1,
    'roses': 2,
    'sunflowers': 3,
    'tulips': 4
}
print(label_dict['daisy'])
print(label_dict['dandelion'])
print(label_dict['roses'])
print(label_dict['sunflowers'])
print(label_dict['tulips'])

0
1
2
3
4


In [11]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y
    print(self.__len__())

  def __len__(self):
    return len(self.x)

  def __getitem__(self, index):
    return self.x[index], label_dict[self.y[index]]

In [12]:
train_images = np.array(train_images)
valid_images = np.array(valid_images)
train_images = train_images.transpose((0, 3, 1, 2))
valid_images = valid_images.transpose((0, 3, 1, 2))
train_images.shape

(2055, 3, 180, 180)

In [13]:
train_images = (np.array(train_images) / 255.0).astype(np.float32)
valid_images = (np.array(valid_images) / 255.0).astype(np.float32)
train_images = torch.tensor(train_images)
valid_images = torch.tensor(valid_images)
print(train_images.shape)

torch.Size([2055, 3, 180, 180])


In [14]:
train_dataset = Dataset(train_images, train_df['Label'])
valid_dataset = Dataset(valid_images, valid_df['Label'])

2055
514


In [15]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)

In [16]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, num_classes=5):
        super(Model, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 45 * 45, 128),
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout 확률을 0.2로 설정
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


In [17]:
import torch.nn as nn
import torchvision.models as models

class ResNet50WithCustomHead(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet50WithCustomHead, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        in_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.resnet50(x)  # ResNet-50의 전체 순전파를 수행
        return x


In [18]:
# 모델 생성
model = ResNet50WithCustomHead(num_classes=5).to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\dustn/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [20]:
best_accuracy = 0.0
best_model_filename = './models/best_model.pt'

In [21]:
def validate(model, data_loader, criterion):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    total_loss = 0.0
    global best_accuracy

    with torch.no_grad():
        for batch_data in data_loader:
            images, labels = batch_data
            images = images.to(device)
            labels = labels.to(device)

            # 순전파
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # 정확도 계산
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    average_loss = total_loss / len(data_loader)

    if accuracy > best_accuracy:
      print("Saving the best model...")
      torch.save(model.state_dict(), best_model_filename)
      best_accuracy = accuracy

    return accuracy, average_loss

In [22]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0.0

    for batch_data in tqdm(train_loader):
        images, labels = batch_data
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 가중치 갱신
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss/len(train_loader)}")

    # 검증 데이터로 정확도 계산
    valid_accuracy, valid_loss = validate(model, valid_loader, criterion)
    print(f"Validation Accuracy: {valid_accuracy:.2f}% - Validation Loss: {valid_loss:.4f}")


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1/20] - Loss: 0.758131795760357
Saving the best model...
Validation Accuracy: 70.23% - Validation Loss: 1.2800


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [2/20] - Loss: 0.5856962185917478
Saving the best model...
Validation Accuracy: 70.43% - Validation Loss: 1.7412


  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch [18/20] - Loss: 0.10118936535648325
Validation Accuracy: 83.27% - Validation Loss: 0.5680


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [19/20] - Loss: 0.048545088738936815
Validation Accuracy: 86.58% - Validation Loss: 0.4424


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [20/20] - Loss: 0.018468869616326905
Validation Accuracy: 85.99% - Validation Loss: 0.6137


In [23]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0.0

    for batch_data in tqdm(train_loader):
        images, labels = batch_data
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 가중치 갱신
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss/len(train_loader)}")

    # 검증 데이터로 정확도 계산
    valid_accuracy, valid_loss = validate(model, valid_loader, criterion)
    print(f"Validation Accuracy: {valid_accuracy:.2f}% - Validation Loss: {valid_loss:.4f}")


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1/20] - Loss: 0.0034747644434294975
Validation Accuracy: 87.16% - Validation Loss: 0.5576


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [2/20] - Loss: 0.0011473321891654368
Validation Accuracy: 87.16% - Validation Loss: 0.5987


  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch [18/20] - Loss: 2.7108462249585003e-05
Validation Accuracy: 87.74% - Validation Loss: 0.6607


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [19/20] - Loss: 2.400733489015362e-05
Validation Accuracy: 87.74% - Validation Loss: 0.6633


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [20/20] - Loss: 2.1308493690715597e-05
Validation Accuracy: 87.74% - Validation Loss: 0.6662
